In [1]:
import os

# Detect if we are in the TIRA sandbox
# Install the required dependencies if we are not in the sandbox.
if 'TIRA_DATASET_ID' not in os.environ:
    !pip3 install python-terrier tira==0.0.88 ir_datasets
else:
    print('We are in the TIRA sandbox.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 KB 412.2 kB/s eta 0:00:0000:01m
  Attempting uninstall: tira
    Found existing installation: tira 0.0.69
    Uninstalling tira-0.0.69:
      Successfully uninstalled tira-0.0.69


In [2]:
# Import the required libraries
print('importing libraries...')
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
ensure_pyterrier_is_loaded()

from load_dataset import load_dataset
from create_index import create_index
from create_model import create_model
from generate_custom_stopwords_improved import generate_custom_stopwords_improved
print('Done. Libraries imported.')

importing libraries...
Due to execution in TIRA, I have patched ir_datasets to always return the single input dataset mounted to the sandbox.
Start PyTerrier with version=5.7, helper_version=0.0.7, no_download=True


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Done. Libraries imported.


In [3]:
from create_index import create_index
from load_dataset import load_dataset

training_dataset = 'ir-lab-jena-leipzig-wise-2023/training-20231104-training'
queries = load_dataset(training_dataset)['queries']

stopword_list = './stopwordlists/standard_english/stopwords_english_long.txt' 

run_name = 'original'
output_dir = 'runs/hello-world-examplss'
run_output_dir = output_dir + '/' + run_name


config = {'stopwords': stopword_list, 'stemmer': None}

improved_index = create_index(load_dataset(training_dataset)['documents'], config)
print("index created")

improved_model = create_model(improved_index)
print("model created")

run = improved_model(queries)

!rm -Rf {run_output_dir}
!mkdir -p {run_output_dir}

persist_and_normalize_run(run, run_name, run_output_dir)

No settings given in /root/.tira/.tira-settings.json. I will use defaults.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.
Load ir_dataset "ir-lab-jena-leipzig-wise-2023/training-20231104-training" from tira.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.
Load ir_dataset "ir-lab-jena-leipzig-wise-2023/training-20231104-training" from tira.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.
index created
model created
The run file is normalized outside the TIRA sandbo

In [5]:
from trectools import TrecRun, TrecQrel, TrecEval
from tira.rest_api_client import Client
from glob import glob
import pandas as pd
tira = Client()


def load_qrels(dataset):
    return TrecQrel(tira.download_dataset('ir-lab-jena-leipzig-wise-2023', dataset, truth_dataset=True) + '/qrels.txt')

def evaluate_run(qrels, runFile):
    run = TrecRun(runFile)
    trec_eval = TrecEval(run, qrels)

    return {
        'run': run.get_runid(),
        'nDCG@10': trec_eval.get_ndcg(depth=10),
        'nDCG@10 (unjudgedRemoved)': trec_eval.get_ndcg(depth=10, removeUnjudged=True),
        'MAP': trec_eval.get_map(depth=10),
        'MRR': trec_eval.get_reciprocal_rank(),
        'P@10': trec_eval.get_precision(depth=10),
        'P': trec_eval.get_precision()
    }

def test_model(runFile):
    training_qrels = load_qrels('training-20231104-training')

    print("Overall performance:\n")
    print(evaluate_run(training_qrels, runFile))
    print("\n")

result = []

result = evaluate_run(load_qrels('training-20231104-training'), 'runs/hello-world-examplss/original/run.txt')
print(result)


No settings given in /root/.tira/.tira-settings.json. I will use defaults.
No settings given in /root/.tira/.tira-settings.json. I will use defaults.


/usr/local/lib/python3.10/dist-packages/trectools/trec_eval.py:294: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


{'run': 'original', 'nDCG@10': 0.17477708457067012, 'nDCG@10 (unjudgedRemoved)': 0.5296970226402502, 'MAP': 0.11660141778037744, 'MRR': 0.26498239884607894, 'P@10': 0.09150521609538004, 'P': 0.0032742175856929957}
